<a href="https://colab.research.google.com/github/cagBRT/PySpark/blob/master/PySpark_3_DelayedFlights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparing data for Machine Learning Models
This notebook uses a dataset of airplane flights. The dataset has 5.8million rows.<br>

This notebook demonstrates some of the commands that are used to prepare the data for a machine learning model. 

This notebook requires a large dataset be loaded onto your Google Drive because it is too large for the github cloned repo. 

1. Go to this [kaggle link](https://www.kaggle.com/code/miquar/explore-flights-csv-airports-csv-airlines-csv/data?select=flights.csv)<br>

2. Download the data by clicking on the Download button in the top right corner. 

3. This will download to your local computer a zip folder called archive.zip

4. Open the folder by clicking on the archive folder

5. Go to your Google drive and upload flights.csv by doing the following: 
>a. In the upper left corner click on "New"<br>
b. Click on Upload file<br>
c. On your local machine find the archive folder that you opened<br>
d. Select the file called flights.csv<br>
 This is a large file (565MB) so it will take a few minutes to upload. 

Once you see the file in your Google Drive, you are ready to start working with this notebook.


**Mount your Google Drive to this notebook**

In [ ]:
# Point Colaboratory to your Google Drive
from google.colab import drive
drive.mount('/gdrive')

Distributed Computing<br>



https://medium.com/@rmache/big-data-with-spark-in-google-colab-7c046e24b3


https://medium.com/grabngoinfo/install-pyspark-3-on-google-colab-the-easy-way-577ec4a2bcd8

Check for the latest version of Spark at https://pypi.org/project/pyspark/#history

https://www.transtats.bts.gov/OT_Delay/OT_DelayCause1.asp


https://grabngoinfo.com/install-pyspark-3-on-google-colab-the-easy-way/



In [ ]:
!git clone https://github.com/cagBRT/PySpark.git

In [ ]:
!pip install pyspark

# Create an Entry Point to PySpark
<br>
SparkSession starts the PySpark Session. 
<br>
This begins the ability to program with RDD (Resilient Data Distribution), DataFrame, and DataSet

In [ ]:
#Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
#getOrCreate gets or creates a session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
spark

In [ ]:
#Import a Spark function from library
from pyspark.sql.functions import col

In [ ]:
import os
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

#import findspark
#findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

print("If no error - everything is working")

from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [ ]:
# Tools we need to connect to the Spark server, load our data,
# clean it and prepare it
from pyspark import SparkContext
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.sql.functions import isnan, when, count, col
# Set up constants
CSV_2007= "/content/gdrive/My Drive/Colab Datasets/2007.csv.bz2" 
CSV_2008= "/content/gdrive/My Drive/Colab Datasets/2008.csv.bz2"
APP_NAME = "Flight Delays"
SPARK_URL = "local[*]"
RANDOM_SEED = 141109
TRAINING_DATA_RATIO = 0.7
RF_NUM_TREES = 8
RF_MAX_DEPTH = 4
RF_NUM_BINS = 32


We need to mount the Gdrive because the dataset is in three files and one of them is quite large. <bR>
The cloned repo contains two of the files:<Br>
"/content/cloned-repo/airports.csv"<br>
"/content/cloned-repo/airlines.csv"<br>
<br>
The third file needs to be loaded onto your GDrive and then the program can read it from there. 


In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/cagBRT/PySpark.git cloned-repo
#%cd cloned-repo
#!ls

Get the data

In [ ]:
# Tools we need to connect to the Spark server, load our data,
# clean it and prepare it
from pyspark import SparkContext
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import isnan, when, count, col

In [ ]:
# Set up constants
FLIGHTS= "/gdrive/MyDrive/flights.csv" 
AIRPORTS= "/content/cloned-repo/airports.csv"
AIRLINES= "/content/cloned-repo/airlines.csv"
APP_NAME = "Flight Delays"
SPARK_URL = "local[*]"
RANDOM_SEED = 141109
TRAINING_DATA_RATIO = 0.7
RF_NUM_TREES = 8
RF_MAX_DEPTH = 4
RF_NUM_BINS = 32

A Spark session has to be started. 

In [ ]:
# Connect to the Spark server
spark = SparkSession.builder.appName(APP_NAME).master(FLIGHTS).getOrCreate()

The dataset is 5.8 milllion rows of data. <br>
It will take a minute to load it into the Spark session

In [ ]:
# Load datasets
flights_df = spark.read.options(header="true",inferschema = "true").csv(FLIGHTS)

This next command can take a while to execute, be patient or skip it. 

In [ ]:
flights_df.describe().show()

The dataset is 5.8million rows by 31 columns. <br>
If your dataset is not 5819079 rows by 31 columns, you may not have uploaded the data properly. 

We can see our dataset is unbalanced. We have way more "Not cancelled" flights than "cancelled" flights.

In [ ]:
datacount=flights_df.groupBy("CANCELLED").count()
datacount.show()

In [ ]:
print(f"The shape is {flights_df.count():d} rows by {len(flights_df.columns):d} columns.")

The dataset has a lot of null values (missing values)<br>
30 million missing values

In [ ]:
null_counts = flights_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c)for c in flights_df.columns]).toPandas().to_dict(orient='records')
print(f"We have {sum(null_counts[0].values()):d} null values in this dataset.")

The list of columns and data types shows there are string value columns. If we are going to use this data we need to either change the strings to values the model can ingest or remove the columns.

In [ ]:
flights_df.printSchema()

In [ ]:
flights_df.dtypes

Let's drop the CANCELLATION_REASON column

In [ ]:
flights_df = flights_df.drop(flights_df.CANCELLATION_REASON)
flights_df = flights_df.na.drop()

In [ ]:
flights_df.dtypes

Select from the "CANCELLED" column only unique values. <br>
We select to keep only the rows where the flight was not cancelled<br>

1= flight cancelled<br>

0=flight not cancelled

We want the model to predict if a flight will be cancelled. 

In [ ]:
#flights_df.select('CANCELLED').distinct().rdd.map(lambda r: r[0]).collect()
#flights_df

Our features for the model will be only those inputs that will halp make a prediction. 

In [ ]:
feature_cols = ['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'DEPARTURE_TIME', 'ARRIVAL_DELAY', 'FLIGHT_NUMBER', 'DISTANCE', 'DIVERTED']

In [ ]:
feature_cols

VectorAssembler = A feature transformer that merges multiple columns into a vector column.<br>
Next we add the features column to the dataset

In [ ]:
flights_df = VectorAssembler(inputCols=feature_cols, outputCol="features").transform(flights_df)

In [ ]:
datacount=flights_df.groupBy("CANCELLED").count()
datacount.show()

The features column is at the end of the columns list

In [ ]:
flights_df.show()

**Creating a subset of the data**

We can create a subset of the dataset, selecting only those flights that were cancelled

In [ ]:
flights_cancelled.select("Cancelled", "features").show(20)

Selecting columns will get us a subset of the dataset

In [ ]:
flights_subset.select("Origin_airport","Destination_airport","Airline","Diverted").show(15)

In [ ]:
df=flights_df.select("Cancelled", "features").show(5)